The OSM is connected to clk_wiz/locked to avoid being removed during implementation. 
![](./diagram.PNG)
![](./timing.PNG)
As shown in the timing report, the critical path ends with `zcu104_fpx_i/FPDiv_0/inst/fR0_d1_reg[28]/D`, and its input net is `zcu104_fpx_i/FPDiv_0/inst/minusOp[28]`.The source of `minusOp[28]` is `zcu104_fpx_i/FPDiv_0/inst/SelFunctionTable7/fR0_d1_reg[28]_i_1`. Note that `minusOp[28]` has another name `zcu104_fpx_i/FPDiv_0/inst/SelFunctionTable7/D[9]` if seen from lower hierachy.
![](./net1.PNG)
![](./net2.PNG)


In [1]:
import os
import re
import subprocess
from com.xilinx.rapidwright.debug import OSM
from com.xilinx.rapidwright.design import Design
from com.xilinx.rapidwright.util import FileTools
from com.xilinx.rapidwright.design import NetType
from com.xilinx.rapidwright.design import Unisim
from com.xilinx.rapidwright.design import SitePinInst
from com.xilinx.rapidwright.device import Device
from com.xilinx.rapidwright.edif import EDIFCell
from com.xilinx.rapidwright.edif import EDIFCellInst
from com.xilinx.rapidwright.edif import EDIFDirection
from com.xilinx.rapidwright.edif import EDIFNet
from com.xilinx.rapidwright.edif import EDIFNetlist
from com.xilinx.rapidwright.edif import EDIFPort
from com.xilinx.rapidwright.edif import EDIFTools
from com.xilinx.rapidwright.edif import EDIFHierCellInst
from com.xilinx.rapidwright.edif import EDIFHierPortInst
from com.xilinx.rapidwright.edif import EDIFPortInst
from com.xilinx.rapidwright.router import Router

from com.xilinx.rapidwright.timing import TimingEdge
from com.xilinx.rapidwright.timing import TimingManager
from com.xilinx.rapidwright.timing import TimingVertex
import org.jgrapht.GraphPath
from java.util import Collections
from java.util import HashMap
from java.util import ArrayList
from pprint import pprint

In [2]:
design = Design.readCheckpoint("osmlock_routed.dcp")
zcu104_fpx_i = EDIFHierCellInst("zcu104_fpx_i",design.getNetlist().getCellInstFromHierName("zcu104_fpx_i"))

get OSM Probe

In [3]:
OSM_Probe = []
OSM_Array = design.getNetlist().getCellInstFromHierName("zcu104_fpx_i/OSM_Array_0")
for PortInst in OSM_Array.getPortInsts():
    if "D" in PortInst.getName():
        OSM_Probe.append(PortInst)
print(OSM_Probe)

[OSM_Array_0/D[0]]


get the OSM cell, each OSM is composed of a single register

In [4]:
OSM_Cell = []
for i in OSM_Array.getCellType().getAllLeafDescendants(OSM_Array.getCellName()):
    if i.getCellType().getName() == "FDCE":
        cell_name = "zcu104_fpx_i/OSM_Array_0" + i.getFullHierarchicalInstName()[i.getFullHierarchicalInstName().find('/'):]
        OSM_Cell.append(design.getCell(cell_name))
print(OSM_Cell)

[zcu104_fpx_i/OSM_Array_0/inst/generate_osm_array[0].osm_inst/Q1_reg(BEL: DFF2)]


get the OSM's sitePin

In [5]:
OSM_Port = []
OSM_SitePin = []
OSM_Port.append(OSM_Cell[0].getEDIFCellInst().getPortInst("C"))
siteWires = []
OSM_SitePin.append(OSM_Cell[0].getSitePinFromPortInst(OSM_Cell[0].getEDIFCellInst().getPortInst("C"), siteWires))
print(OSM_Port, OSM_SitePin)

([Q1_reg/C], [IN SLICE_X51Y237.CLK1])


remove the OSM from the orignial logical and physical net, note that the OSM are connected to clk_wiz/locked to avoid being removed 

In [6]:
OSMLogicNet = OSM_Probe[0].getNet()
OSMPhysNet = design.getNetlist().getPhysicalNetFromPin("zcu104_fpx_i", OSM_Probe[0], design)
OSMPhysNet.removePin(OSM_SitePin[0])
OSMPhysNet.unroute()
OSMLogicNet.removePortInst(OSM_Probe[0])

OSM_Array_0/D[0]

get the critical path's terminal register

In [7]:
T0_EDIFHierPortInst = design.getNetlist().getHierPortInstFromName("zcu104_fpx_i/FPDiv_0/inst/fR0_d1_reg[28]/D") # T means terminal
T0_Cell = design.getCell("zcu104_fpx_i/FPDiv_0/inst/fR0_d1_reg[28]")
T0_SiteWire = T0_Cell.getSiteWireNameFromLogicalPin("D")
T0_Net = T0_Cell.getSiteInst().getNetFromSiteWire(T0_SiteWire)
print(T0_Net)

zcu104_fpx_i/FPDiv_0/inst/SelFunctionTable7/D[9]


I first tried to add the OSM's port and pin to the net to be monitored in both logical level and phyiscal level

In [8]:
T0_Net.getLogicalNet().addPortInst(OSM_Port[0])
T0_Net.addPin(OSM_SitePin[0]) 
print(T0_Net.getLogicalNet().getPortInsts(),T0_Net.getPins())

([fR0_d1_reg[28]_i_1/O[1], D[9], Q1_reg/C], [IN SLICE_X51Y237.CLK1])


In [9]:
design.writeCheckpoint("updated_osm.dcp")

However, the saved dcp file cannot be opened by Vivado
![](./error.PNG)
As a result, I tried to create a new logical net and a new physical net and add Ports/Pins to them. 

In [8]:
OSM0_EDIFNet = OSM_Array.getParentCell().createNet("OSM0")
OSM0_EDIFNet.addPortInst(OSM_Probe[0])
EDIFTools.connectDebugProbe(OSM0_EDIFNet,"zcu104_fpx_i/FPDiv_0/inst/SelFunctionTable7/D[9]", "OSM0", zcu104_fpx_i, design.getNetlist(), None)

In [9]:
OSM0_Net = design.createNet(OSM0_EDIFNet)
OSM0_Net.addPin(OSM_SitePin[0])
OSM0_Net.addPins(T0_Net.getPins())
print(OSM0_Net.getPins())

[IN SLICE_X51Y237.CLK1]


However, the `T0_Net` is an intra-site net, and does not have any Pin. So `OSM0_Net.addPins(T0_Net.getPins())` does not actually connect the origin net with the new OSM net together.